# build obs_vista camera description

In ths notebook we will build the fits files describing the VISTA camera. This sis a hack. Going forward to the gen 3 Butler I think we need to specify teh Camera in a yml file and use

https://github.com/lsst/obs_base/blob/master/python/lsst/obs/base/yamlCamera.py

to build the camera.

For stacks the gain is effectively multiplied by the number of exposures (6). However there are complications due to clipping.

http://casu.ast.cam.ac.uk/surveys-projects/vista/technical/vista-gain

## 1 Make ccd fits files
the example code does not run because the LSST stack has been changed for the latest version 20.0.0 This needs to be rewritten. It will also allow us to start to see how to interact with the LSST stack from within Python.

In [1]:
import yaml
import lsst
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import lsst.geom as geom
import numpy as np

from astropy.io import fits
from astropy.table import Table

In [30]:
for n in [3,6,9,12,15]:
    print(1.024**n)

1.073741824
1.152921504606847
1.2379400392853805
1.3292279957849162
1.4272476927059603


In [2]:
gain = Table.read('./gain.csv')
readout_noise = Table.read('./readout_noise.csv')
saturation = Table.read('./saturation.csv')

In [3]:
gain #electrons/ADU

id,NB118,Z,Y,J,Average
int64,float64,float64,float64,float64,float64
1,3.648,3.628,3.63,3.717,3.66
2,4.28,4.251,4.206,4.247,4.25
3,3.957,3.947,3.858,4.045,3.95
4,4.154,4.102,4.041,4.335,4.16
5,4.16,4.139,4.125,4.517,4.24
6,4.063,4.074,4.11,4.179,4.11
7,3.882,3.78,3.787,3.903,3.84
8,4.153,4.216,4.176,4.326,4.22
9,4.639,4.5,4.396,4.589,4.53


In [4]:
readout_noise # ADU?

id,NB118,Z,Y,J
int64,float64,float64,float64,float64
1,22.903,22.782,22.791,27.231
2,22.394,26.694,22.007,26.665
3,20.708,24.783,20.186,25.402
4,21.734,25.758,21.145,27.222
5,21.77,25.989,21.586,28.364
6,21.258,25.579,21.504,26.241
7,24.378,23.737,19.814,24.505
8,21.732,26.474,21.853,27.16
9,19.418,18.838,18.401,19.209


In [5]:
saturation #ADU? non-linearity - percent

detector,saturation,non-linearity
int64,int64,float64
1,33000,2.2
2,32000,3.3
3,33000,3.8
4,32000,3.5
5,24000,2.0
6,26000,3.0
7,35000,2.0
8,33000,3.4
9,35000,3.3


In [6]:
#hsc_cam = fits.open('/Users/rs548/GitHub/lsstsw/stack/1a1d771/DarwinX86/obs_subaru/19.0.0-66-gd8347dd2+3/hsc/camera/0_00.fits')

In [7]:
from astropy.table import Table
hsc_t = Table.read('./hsc_example.fits')

In [8]:
hscAfw = afwTable.BaseCatalog.readFits('./hsc_example.fits')

In [9]:
vistaAfw = hscAfw[:1]


In [10]:
hsc_t

flags [3],name,bbox_min_x,bbox_min_y,bbox_extent_x,bbox_extent_y,gain,saturation,suspectlevel,readnoise,readoutcorner,linearity_coeffs [4],linearity_type,raw_bbox_min_x,raw_bbox_min_y,raw_bbox_extent_x,raw_bbox_extent_y,raw_databbox_min_x,raw_databbox_min_y,raw_databbox_extent_x,raw_databbox_extent_y,raw_xyoffset_x,raw_xyoffset_y,raw_horizontaloverscanbbox_min_x,raw_horizontaloverscanbbox_min_y,raw_horizontaloverscanbbox_extent_x,raw_horizontaloverscanbbox_extent_y,raw_verticaloverscanbbox_min_x,raw_verticaloverscanbbox_min_y,raw_verticaloverscanbbox_extent_x,raw_verticaloverscanbbox_extent_y,raw_prescanbbox_min_x,raw_prescanbbox_min_y,raw_prescanbbox_extent_x,raw_prescanbbox_extent_y
,,pix,pix,pix,pix,electron adu^-1,adu,adu,electron,,,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
bool,bytes64,int32,int32,int32,int32,float64,float64,float64,float64,int32,float64,bytes64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32
True .. False,0,0,0,512,4176,4.2097,44000.0,50000.0,1.36,0,1.48408e-07 .. nan,Squared,0,0,536,4241,8,49,512,4176,0,0,520,49,16,4176,8,4225,512,16,8,0,512,49
True .. False,1,512,0,512,4176,4.054,44000.0,50000.0,1.43,1,1.61081e-07 .. nan,Squared,536,0,536,4241,552,49,512,4176,0,0,536,49,16,4176,552,4225,512,16,552,0,512,49
True .. False,2,1024,0,512,4176,3.9717,44000.0,50000.0,1.46,0,1.70965e-07 .. nan,Squared,1072,0,536,4241,1080,49,512,4176,0,0,1592,49,16,4176,1080,4225,512,16,1080,0,512,49
True .. False,3,1536,0,512,4176,3.9878,44000.0,50000.0,1.43,1,2.00942e-07 .. nan,Squared,1608,0,536,4241,1624,49,512,4176,0,0,1608,49,16,4176,1624,4225,512,16,1624,0,512,49


In [11]:
hsc_t['flags'][0]

array([ True, False, False])

In [12]:
hsc_t['linearity_coeffs'][0]

array([1.48408e-07, 0.00000e+00,         nan,         nan])

## Simplest possible hack

Lets just take the hsc example, replace the values and save the fits files.

In [13]:
vis_t = hsc_t[:1]

In [14]:
vis_t

flags [3],name,bbox_min_x,bbox_min_y,bbox_extent_x,bbox_extent_y,gain,saturation,suspectlevel,readnoise,readoutcorner,linearity_coeffs [4],linearity_type,raw_bbox_min_x,raw_bbox_min_y,raw_bbox_extent_x,raw_bbox_extent_y,raw_databbox_min_x,raw_databbox_min_y,raw_databbox_extent_x,raw_databbox_extent_y,raw_xyoffset_x,raw_xyoffset_y,raw_horizontaloverscanbbox_min_x,raw_horizontaloverscanbbox_min_y,raw_horizontaloverscanbbox_extent_x,raw_horizontaloverscanbbox_extent_y,raw_verticaloverscanbbox_min_x,raw_verticaloverscanbbox_min_y,raw_verticaloverscanbbox_extent_x,raw_verticaloverscanbbox_extent_y,raw_prescanbbox_min_x,raw_prescanbbox_min_y,raw_prescanbbox_extent_x,raw_prescanbbox_extent_y
,,pix,pix,pix,pix,electron adu^-1,adu,adu,electron,,,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
bool,bytes64,int32,int32,int32,int32,float64,float64,float64,float64,int32,float64,bytes64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32
True .. False,0,0,0,512,4176,4.2097,44000.0,50000.0,1.36,0,1.48408e-07 .. nan,Squared,0,0,536,4241,8,49,512,4176,0,0,520,49,16,4176,8,4225,512,16,8,0,512,49


In [15]:
#Change the pixel coord stuff for every ccd
vis_t['bbox_extent_x'] = 2048 #These will have to be changed for each ccd
vis_t['bbox_extent_y'] = 2048
vis_t['raw_bbox_extent_x'] = 2048
vis_t['raw_bbox_extent_y'] = 2048
vis_t['raw_databbox_min_x'] = 0
vis_t['raw_databbox_min_y'] = 0 
vis_t['raw_databbox_extent_x'] = 2048
vis_t['raw_databbox_extent_y'] = 2048
vis_t['raw_horizontaloverscanbbox_min_x'] = 0
vis_t['raw_horizontaloverscanbbox_min_y'] = 0
vis_t['raw_horizontaloverscanbbox_extent_x'] = 2048
vis_t['raw_horizontaloverscanbbox_extent_y'] = 2048
vis_t['raw_verticaloverscanbbox_min_x'] = 0
vis_t['raw_verticaloverscanbbox_min_y'] = 2048 #???
vis_t['raw_verticaloverscanbbox_extent_x'] = 2048



vistaAfw['bbox_extent_x'] = 2048 #These will have to be changed for each ccd
vistaAfw['bbox_extent_y'] = 2048
vistaAfw['raw_bbox_extent_x'] = 2048
vistaAfw['raw_bbox_extent_y'] = 2048
vistaAfw['raw_databbox_min_x'] = 0
vistaAfw['raw_databbox_min_y'] = 0 
vistaAfw['raw_databbox_extent_x'] = 2048
vistaAfw['raw_databbox_extent_y'] = 2048
vistaAfw['raw_horizontaloverscanbbox_min_x'] = 0
vistaAfw['raw_horizontaloverscanbbox_min_y'] = 0
vistaAfw['raw_horizontaloverscanbbox_extent_x'] = 2048
vistaAfw['raw_horizontaloverscanbbox_extent_y'] = 2048
vistaAfw['raw_verticaloverscanbbox_min_x'] = 0
vistaAfw['raw_verticaloverscanbbox_min_y'] = 2048 #???
vistaAfw['raw_verticaloverscanbbox_extent_x'] = 2048

In [16]:
vis_t

flags [3],name,bbox_min_x,bbox_min_y,bbox_extent_x,bbox_extent_y,gain,saturation,suspectlevel,readnoise,readoutcorner,linearity_coeffs [4],linearity_type,raw_bbox_min_x,raw_bbox_min_y,raw_bbox_extent_x,raw_bbox_extent_y,raw_databbox_min_x,raw_databbox_min_y,raw_databbox_extent_x,raw_databbox_extent_y,raw_xyoffset_x,raw_xyoffset_y,raw_horizontaloverscanbbox_min_x,raw_horizontaloverscanbbox_min_y,raw_horizontaloverscanbbox_extent_x,raw_horizontaloverscanbbox_extent_y,raw_verticaloverscanbbox_min_x,raw_verticaloverscanbbox_min_y,raw_verticaloverscanbbox_extent_x,raw_verticaloverscanbbox_extent_y,raw_prescanbbox_min_x,raw_prescanbbox_min_y,raw_prescanbbox_extent_x,raw_prescanbbox_extent_y
,,pix,pix,pix,pix,electron adu^-1,adu,adu,electron,,,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
bool,bytes64,int32,int32,int32,int32,float64,float64,float64,float64,int32,float64,bytes64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32
True .. False,0,0,0,2048,2048,4.2097,44000.0,50000.0,1.36,0,1.48408e-07 .. nan,Squared,0,0,2048,2048,0,0,2048,2048,0,0,0,0,2048,2048,0,2048,2048,16,8,0,512,49


In [17]:
for i in range(16):
    #from astropy table
    #vis_t.write('../camera/{}.fits'.format(i))
    #from afwCatalog
    #every ccd called 0
    #vis_t['name'] = i
    #vistaAfw['name'] = bytes(i)
    vis_t['gain'] = gain[i]['Average'] *6 #This should be enforced according to number in header
    vistaAfw['gain'] = gain[i]['Average'] *6
    vis_t['linearity_coeffs'] = np.array([saturation[i]['non-linearity']/100,0,np.nan,np.nan]) #percentages
    vistaAfw['linearity_coeffs'] = np.array([saturation[i]['non-linearity']/100,0,np.nan,np.nan])
    vis_t['saturation'] = saturation[i]['saturation']
    vistaAfw['saturation'] = saturation[i]['saturation']
    vis_t['readnoise'] =    np.mean([readout_noise[i]['Z'],readout_noise[i]['Y'],readout_noise[i]['J']])
    vistaAfw.writeFits('../camera/{}.fits'.format(i+1))

In [18]:
vis_t

flags [3],name,bbox_min_x,bbox_min_y,bbox_extent_x,bbox_extent_y,gain,saturation,suspectlevel,readnoise,readoutcorner,linearity_coeffs [4],linearity_type,raw_bbox_min_x,raw_bbox_min_y,raw_bbox_extent_x,raw_bbox_extent_y,raw_databbox_min_x,raw_databbox_min_y,raw_databbox_extent_x,raw_databbox_extent_y,raw_xyoffset_x,raw_xyoffset_y,raw_horizontaloverscanbbox_min_x,raw_horizontaloverscanbbox_min_y,raw_horizontaloverscanbbox_extent_x,raw_horizontaloverscanbbox_extent_y,raw_verticaloverscanbbox_min_x,raw_verticaloverscanbbox_min_y,raw_verticaloverscanbbox_extent_x,raw_verticaloverscanbbox_extent_y,raw_prescanbbox_min_x,raw_prescanbbox_min_y,raw_prescanbbox_extent_x,raw_prescanbbox_extent_y
,,pix,pix,pix,pix,electron adu^-1,adu,adu,electron,,,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
bool,bytes64,int32,int32,int32,int32,float64,float64,float64,float64,int32,float64,bytes64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32
True .. False,0,0,0,2048,2048,29.759999999999998,34000.0,50000.0,19.869666666666664,0,0.033 .. nan,Squared,0,0,2048,2048,0,0,2048,2048,0,0,0,0,2048,2048,0,2048,2048,16,8,0,512,49


## Gen 3 YAML file version

In gen 3 the camera is defined in a single yaml file. Since we are currently developing obs_vista to be compatible with both we here also make the yaml file. gen 2 will be deprecated fully at some point before LSST commences.

In [19]:
camera = {
    'name': 'vista',
    'plateScale':1.24,
    'transforms': {
        'nativeSys': 'FocalPlane',
        'FieldAngle': {
            'transformType': 'radial',
            'coeffs': [0.0, 1.0]
        }
    },
    'CCDs': {}
    
}
    
    


In [26]:
for i in range(16):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [27]:
for i in range(16):
    this_dict={
        'name': 'ccd{}'.format(i+1),
        'id': i+1,
        'serial': str(i+1),
        'detectorType': 0,
        'bbox': [[0,0], [2048, 2048]],
        'pixelSize': [1.,1.],
        'transformDict':{
            'nativeSys': 'Pixels',
            'transforms': 'None'
        },
        'refpos': [0,0],
        'offset': [0,0],
        'transposeDetector': False,
        'pitch': 0.0,
        'yaw': 0.0,
        'roll': 0.0,
        'amplifiers':{
            'AMP_E2V':{
                'rawBBox': [[0,0],[2048, 2048]],
                'hdu': i ,
                'ixy': [0,0],
                'perAmpData': True,
                'rawDataBBox': [[0,0],[2048, 2048]],
                'rawSerialOverscanBBox': [[0,0],[0, 0]],
                'rawParallelOverscanBBox': [[0,0],[0, 0]],
                'rawSerialPrescanBBox': [[0,0],[0, 0]],
                'readCorner': 'LL',
                'gain': float(gain[i]['Average'] * 6),
                'readNoise': float(
                    np.mean([
                        readout_noise[i]['Z'],
                        readout_noise[i]['Y'],
                        readout_noise[i]['J']
                    ])
                ),
                'saturation': int(saturation[i]['saturation']),
                'flipXY': [False, False],
                'linearityCoeffs': [
                    float(saturation[i]['non-linearity']/100), 
                    int(saturation[i]['saturation'])
                ],
                'linearityType': "None",
                'linearityThreshold': 0,
                'linearityMax': int(saturation[i]['saturation']),
            }
        }
    }
    camera['CCDs']['ccd{}'.format(i+1)]=this_dict

In [28]:
camera

{'name': 'vista',
 'plateScale': 1.24,
 'transforms': {'nativeSys': 'FocalPlane',
  'FieldAngle': {'transformType': 'radial', 'coeffs': [0.0, 1.0]}},
 'CCDs': {'ccd1': {'name': 'ccd1',
   'id': 1,
   'serial': '1',
   'detectorType': 0,
   'bbox': [[0, 0], [2048, 2048]],
   'pixelSize': [1.0, 1.0],
   'transformDict': {'nativeSys': 'Pixels', 'transforms': 'None'},
   'refpos': [0, 0],
   'offset': [0, 0],
   'transposeDetector': False,
   'pitch': 0.0,
   'yaw': 0.0,
   'roll': 0.0,
   'amplifiers': {'AMP_E2V': {'rawBBox': [[0, 0], [2048, 2048]],
     'hdu': 0,
     'ixy': [0, 0],
     'perAmpData': True,
     'rawDataBBox': [[0, 0], [2048, 2048]],
     'rawSerialOverscanBBox': [[0, 0], [0, 0]],
     'rawParallelOverscanBBox': [[0, 0], [0, 0]],
     'rawSerialPrescanBBox': [[0, 0], [0, 0]],
     'readCorner': 'LL',
     'gain': 21.96,
     'readNoise': 24.268,
     'saturation': 33000,
     'flipXY': [False, False],
     'linearityCoeffs': [0.022000000000000002, 33000],
     'linearity

In [29]:


with open('vircam.yaml', 'w') as file:
    documents = yaml.dump(camera, file)